## Imports

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures 

In [3]:
train = pd.read_csv('../datasets/train.csv')

kaggle = pd.read_csv('../datasets/test 2.csv')

train.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,square_footage,bldg_type_1Fam,bldg_type_2fmCon,bldg_type_Duplex,bldg_type_Twnhs,bldg_type_TwnhsE,kitchen_qual_Ex,kitchen_qual_Fa,kitchen_qual_Gd,kitchen_qual_TA
0,109,533352170,60,RL,0.0,13517,Pave,IR1,Lvl,AllPub,...,2204.0,1,0,0,0,0,0,0,1,0
1,544,531379050,60,RL,43.0,11492,Pave,IR1,Lvl,AllPub,...,3035.0,1,0,0,0,0,0,0,1,0
2,153,535304180,20,RL,68.0,7922,Pave,Reg,Lvl,AllPub,...,2114.0,1,0,0,0,0,0,0,1,0
3,318,916386060,60,RL,73.0,9802,Pave,Reg,Lvl,AllPub,...,1828.0,1,0,0,0,0,0,0,0,1
4,255,906425045,50,RL,82.0,14235,Pave,IR1,Lvl,AllPub,...,2121.0,1,0,0,0,0,0,0,0,1


## EDA and Cleaning

In [6]:
train['saleprice'].mean()

181534.626159102

In [7]:
train.shape

(2049, 84)

In [1]:
#pd.set_option('display.max_rows', 500)
#train.isnull().sum().sort_values(ascending = False)

In [9]:
train.columns = [column.replace(' ', '_').lower() for column in train.columns]

train.head(2)

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,square_footage,bldg_type_1fam,bldg_type_2fmcon,bldg_type_duplex,bldg_type_twnhs,bldg_type_twnhse,kitchen_qual_ex,kitchen_qual_fa,kitchen_qual_gd,kitchen_qual_ta
0,109,533352170,60,RL,0.0,13517,Pave,IR1,Lvl,AllPub,...,2204.0,1,0,0,0,0,0,0,1,0
1,544,531379050,60,RL,43.0,11492,Pave,IR1,Lvl,AllPub,...,3035.0,1,0,0,0,0,0,0,1,0


In [10]:
#train.drop(['pool_qc', 'misc_feature', 'alley', 'fence', 'fireplace_qu'], axis = 1, inplace=True)

Considering that the dataset only has 2051 rows there is no point in keeping columns that are missing almost the entire dataset. So for everything above 1000. I am going to drop all columns that are missing 1000 and above. 

In [11]:
train.shape

(2049, 84)

To get my features with high correlation we have to clean the null values. These ones would be Garage Area, Garage Cars, Total Bsmt SF, BsmitFin SF 1

In [12]:
#train.dropna(subset=['garage_area', 'garage_cars', 'total_bsmt_sf', 'bsmtfin_sf_1'], inplace=True)

In [13]:
train.corr()[['saleprice']].sort_values(by = 'saleprice', ascending = False)

/var/folders/np/hm339m411_30q030vqxtjdy80000gn/T/ipykernel_74496/4189176062.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()[['saleprice']].sort_values(by = 'saleprice', ascending = False)


,saleprice
saleprice,1.000000
overall_qual,0.800028
square_footage,0.779302
gr_liv_area,0.697032
garage_area,0.650097
garage_cars,0.648007
total_bsmt_sf,0.628894
1st_flr_sf,0.618345
year_built,0.571726
kitchen_qual_ex,0.551320


In [15]:
train.dtypes

id                    int64
pid                   int64
ms_subclass           int64
ms_zoning            object
lot_frontage        float64
lot_area              int64
street               object
lot_shape            object
land_contour         object
utilities            object
lot_config           object
land_slope           object
neighborhood         object
condition_1          object
condition_2          object
house_style          object
overall_qual          int64
overall_cond          int64
year_built            int64
year_remod/add        int64
roof_style           object
roof_matl            object
exterior_1st         object
exterior_2nd         object
mas_vnr_type         object
mas_vnr_area        float64
exter_qual           object
exter_cond           object
foundation           object
bsmt_qual            object
bsmt_cond            object
bsmt_exposure        object
bsmtfin_type_1       object
bsmtfin_sf_1        float64
bsmtfin_type_2       object
bsmtfin_sf_2        

In [18]:
#Square footage

train['square_footage'] = train['1st_flr_sf'] + train['2nd_flr_sf'] + train['low_qual_fin_sf'] + train['total_bsmt_sf']

print(train[['1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf', 'total_bsmt_sf', 'square_footage']])

      1st_flr_sf  2nd_flr_sf  low_qual_fin_sf  total_bsmt_sf  square_footage
0            725         754                0          725.0          2204.0
1            913        1209                0          913.0          3035.0
2           1057           0                0         1057.0          2114.0
3            744         700                0          384.0          1828.0
4            831         614                0          676.0          2121.0
...          ...         ...              ...            ...             ...
2044        1728           0                0         1884.0          3612.0
2045         861           0                0          861.0          1722.0
2046        1172         741                0          896.0          2809.0
2047        1200           0                0         1200.0          2400.0
2048        1028         776                0          994.0          2798.0

[2049 rows x 5 columns]


Replace all the NaN values with 0.

In [19]:
train.fillna(0,inplace = True)

In [1]:
#pd.set_option('display.max_rows', 500)
#train.isnull().sum().sort_values(ascending = False)

In [48]:
#Save the changes
#train.to_csv('./datasets/train.csv', index = False)